In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv')
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv')
test.head()

In [ ]:
sample=pd.read_csv('/kaggle/input/dont-overfit-ii/sample_submission.csv')
sample.head()

In [ ]:
train.isnull().any().any()

In [ ]:
test.isnull().any().any()

In [ ]:
print('Distributions of first 28 columns')
plt.figure(figsize=(26, 24))
for i, col in enumerate(list(train.columns)[2:30]):
    plt.subplot(7, 4, i + 1)
    plt.hist(train[col])
    plt.title(col)


In [ ]:
corrs = train.corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corrs = corrs[corrs['level_0'] != corrs['level_1']]
corrs.tail(30)

**there is no highly correlated feature as the maximum correlated feature has correlation = 0.338**

In [ ]:
X=train.drop(columns=['id','target'])
y=train['target']
x_test=test.drop(columns=['id'])

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 42)

**try logistic regression model**

In [ ]:
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, y_train)
acc_log = round(logreg.score(x_val,  y_val) * 100, 2)
print('Validation Accuracy =', acc_log)

**try knn model**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn = KNeighborsClassifier(n_neighbors=30)
knn.fit(X_train,y_train)
score = cross_val_score(knn, X_train, y_train, cv=20, scoring='roc_auc')
print('training score = ',score.max())

**try svm**

In [ ]:
from sklearn.svm import SVC
svm = SVC(C=100, kernel='linear', max_iter=100, gamma='auto', probability=True, random_state=0)
svm.fit(X_train, y_train)

In [ ]:
svm_pred = svm.predict_proba(x_test)[:, 1]
df_test = pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv')
submission= pd.DataFrame({'id':np.asarray(df_test.id), 'target':svm_pred})
submission.to_csv("submission.csv", index=False)

In [ ]:
submission